<a href="https://colab.research.google.com/github/kiplimo07/Book_recommendation/blob/main/Book_recomendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-07-27 12:24:00--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   104MB/s    in 0.2s    

2024-07-27 12:24:01 (104 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [26]:
# add your code here - consider creating a new cell for each section of code
df = df_ratings
df.dropna(inplace=True)
df = df[df['isbn'].isin(df_books['isbn'])]
df_ratingCount = (df.
     groupby(by = ['isbn'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'totalRatingCount'})
     [['isbn', 'totalRatingCount']]
    )
rating_with_totalRatingCount = df.merge(df_ratingCount, left_on = 'isbn', right_on = 'isbn', how = 'left')
popularity_threshold = 50
rating_popular_book = rating_with_totalRatingCount.query('totalRatingCount >= @popularity_threshold')
combined = rating_popular_book.merge(df_books, left_on = 'isbn', right_on = 'isbn', how = 'left')
combined = combined.drop_duplicates(['user', 'title'])

user_rating = combined.drop_duplicates(['user', 'title'])
user_rating_pivot = user_rating.pivot(index = 'title', columns = 'user', values = 'rating').fillna(0)
user_rating_matrix = csr_matrix(user_rating_pivot.values)
#The model
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(user_rating_matrix)
#The recommendation
query_index = np.random.choice(user_rating_pivot.shape[0])
distances, indices = model_knn.kneighbors(user_rating_pivot.iloc[query_index, :].values.reshape(1, -1), n_neighbors = 6)

print(f"Recommendations for {user_rating_pivot.index[query_index]}:") # Print the book title for which recommendations are generated
for i in range(1, len(distances.flatten())): # Start from index 1 to skip the book itself
    print(f"{i}. {user_rating_pivot.index[indices.flatten()[i]]} - Distance: {distances.flatten()[i]:.4f}") # Print the recommendation with its distance


Recommendations for Forbidden Fruit:
1. Neanderthal: A Novel - Distance: 0.8531
2. Impulse - Distance: 0.8554
3. The Outsiders - Distance: 0.8617
4. Naked in Death - Distance: 0.8678
5. The Witness - Distance: 0.8694


In [27]:
# prompt: Create a function named get_recommends that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

def get_recommends(book = ""):
  recommended_books = []
  try:
    book_index = user_rating_pivot.index.get_loc(book)
  except KeyError as e:
    print('The given book', e, 'does not exist')
    return recommended_books
  distances, indices = model_knn.kneighbors(user_rating_pivot.iloc[book_index, :].values.reshape(1, -1), n_neighbors = 6)
  for i in range(1, len(distances.flatten())):
      recommended_books.append([user_rating_pivot.index[indices.flatten()[i]], distances.flatten()[i]])
  return recommended_books


In [28]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

[['The Lovely Bones: A Novel', 0.90959823], ['Blue Diary', 0.9101138], ["The Pilot's Wife : A Novel", 0.9172571], ['I Know This Much Is True', 0.9198669], ['The Weight of Water', 0.9210188]]
You haven't passed yet. Keep trying!
